In [2]:
import os
import glob
import sys
import astropy.io.fits
import matplotlib
import numpy as np
import time

import skimage.transform
import cv2
import psycopg2
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from matplotlib.colors import DivergingNorm

from eCallistoProject import plot_config

module_path = os.path.abspath(os.path.join('radiospectra'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import radiospectra

from radiospectra.sources import CallistoSpectrogram

from matplotlib.backends.backend_pdf import PdfPages

import warnings
warnings.filterwarnings("ignore")


In [3]:
def get_db_from_sql():
    engine = create_engine("mysql+pymysql://" + 'root' + ":" 
                           + 'delberin1992' + "@" + 'localhost' 
                           + "/" + 'ecallisto_DB')

    # from sql to DataFrame (pandas)

    SQL_Query = pd.read_sql_query('''SELECT * 
                                  FROM ecallisto_data;''', engine)

    ecallisto_db= pd.DataFrame(SQL_Query)

    return ecallisto_db


get_db_from_sql()

,ID,File_name,Instrument_name,Start_time,End_time
0,1,AUSTRIA-UNIGRAZ_20170906_040000_02.fit.gz,AUSTRIA-UNIGRAZ,2017-09-06 04:00:00,2017-09-06 04:15:00
1,2,ACCIMT-SRI_20170906_001500_59.fit.gz,ACCIMT-SRI,2017-09-06 00:15:00,2017-09-06 00:30:00
2,3,ACCIMT-SRI_20170906_003003_59.fit.gz,ACCIMT-SRI,2017-09-06 00:30:03,2017-09-06 00:45:03
3,4,ACCIMT-SRI_20170906_004508_59.fit.gz,ACCIMT-SRI,2017-09-06 00:45:08,2017-09-06 01:00:08
4,5,ACCIMT-SRI_20170906_010013_59.fit.gz,ACCIMT-SRI,2017-09-06 01:00:13,2017-09-06 01:15:13
...,...,...,...,...,...
2608,2609,ZSTS_20170906_173000_59.fit.gz,ZSTS,2017-09-06 17:30:00,2017-09-06 17:45:00
2609,2610,ZSTS_20170906_174500_59.fit.gz,ZSTS,2017-09-06 17:45:00,2017-09-06 18:00:00
2610,2611,ZSTS_20170906_180000_59.fit.gz,ZSTS,2017-09-06 18:00:00,2017-09-06 18:15:00
2611,2612,ZSTS_20170906_181500_59.fit.gz,ZSTS,2017-09-06 18:15:00,2017-09-06 18:30:00


In [13]:
path = 'R:\\radio\\2002-20yy_Callisto\\2020\\10\\07\\6'

def get_list_of_data_std():    
    ecallisto_db = get_db_from_sql()

    list_of_std =[] 

    for file in ecallisto_db["File_name"]:
        
        full_path = os.path.join(path,file)
        spec = CallistoSpectrogram.read(full_path)

        bgs,_,_,cps = spec.subtract_bg_sliding_window(window_width=200, affected_width=1,
                                                      amount=0.05, change_points = True)
       
        array_std = bgs.data.std()
       
        rfi_rm = bgs.remove_single_freq_rfi(threshold = 8 * array_std, row_window_height=3)
                
        list_of_std.append(rfi_rm.data.std())
        

    return list_of_std


get_list_of_data_std()

FileNotFoundError: [Errno 2] No such file or directory: 'R:\\radio\\2002-20yy_Callisto\\2020\\10\\07\\6\\AUSTRIA-UNIGRAZ_20170906_040000_02.fit.gz'

In [ ]:
def store_std_in_db():    
    engine = create_engine(
        "mysql+pymysql://" + 'root' + ":" + 'delberin1992' + "@" + 'localhost' + "/" + 'ecallisto_DB')

    list_of_std = get_list_of_data_std()

    dataframe_std = pd.DataFrame(list_of_std, columns=['STD'])
    
    ecallisto_db = get_db_from_sql()
    
    ecallisto_db.update(dataframe_std)
    
    ecallisto_db.to_sql('ecallisto', con = engine,if_exists = 'append', chunksize = 5000, index=False) 
    
    
    return ecallisto_db